## Factorization LU

The idea behind LU factorization is instead of manipulating a matrix $A$ we can break the difficulty by factorizing it to a product of a lower triangular matrix $L$ and an upper triangular matrix $U$ such as 
$A = LU$, at least we have 3 benefits from that : 


#### 1 - Solving a system  $Ax = b$ 

As  $A = LU$ instead of solving $Ax = b$ it's equivalent to solve $\begin{cases} Ly = b \\ Ux = y  \end{cases}$ we solve a triangular matrix with $O_(n^2)$ instead of $O_(n^3)$ for Gauss solving method but we obtain $L$ and $U$ with a complexity of $O_(n^3)$

So you will tell me why we do that ?!

we do that when we have to solve the problem for multiple $b$
#### 2 - Computing determinent  $det(A)$ 
$det(A) = det(LU) = det(L) * det(U)$ since $det(T) = \prod_{i=1}^{n} t_{ii} $  when   $T$ a triangular matrix of dimension n

#### 3 -Computing inverse matrix $A^{-1}$
Or when we want to compute the inverse of a square Matrix A, there is a technic where we apply jordan gauss method to $[A|I_n]$ the resulted Matrix is $[I_n | A^{-1}]$

### 1 - Computing $L$ and $U$

In the Gauss method seen just previously, by doing some linear operations on $A$ :

$
A \in M_n(R) \forall i \in \; [k,n] L_i^{(k+1)} = L_i^{(k)} - m_i^{(k)} L_k \tag{1}\label{eq:one}
$ 

where $  m_i^{(k)} = \frac{a_{ik}}{a_{kk}}, k \in [2, n]$    and $a_{kk}$ is called the pivot  



We obtained a triangular upper matrix, and that matrix is actually $U$

And you know that doing linear operations on a matrix is equivalent to mulitiply it by a matrix, since the operation expressed above in $\eqref{eq:one}$ can be reexpressed with matrix this way $A^{k + 1} = G^{(k)} A^{k}$ where 


$G^{(k)} = \begin{pmatrix} 
1 & .. & 0 & 0 & 0 & 0 \\
0 & .. & .. & 0 & 0 & 0\\
0 & .. & 1 & 0 & 0 & 0\\
0 & .. & - m^{(k)}_{k+1} & 1 & 0 \\
0 & .. & .. & 0 & 1 & 0 \\
0 & .. &  - m^{(k)}_{n} & 0 & 0 & 1
\end{pmatrix}$

Wich lead to $U = G^{(n - 1)}..G^{(2)}G^{(1)}A$, but we want a form  $L U =  A$ wihch mean that $L =  (G^{(1)})^{-1}..(G^{(n - 2)})^{-1}(G^{(n - 1)})^{-1}$

To compute $(G^{(k)})^{-1}$ we can use comatrix and det, see [Wikipedia](https://en.wikipedia.org/wiki/Invertible_matrix)

or we can inteligently notice that $G^{(k)} A$ is an operation that Keep $A$ and to each line $L_i$ it does the operation $ L_i  \leftarrow  L_i - m_i^{(k)}  \;| \; i \in \; [k,n]$ so the reverse operation is $ L_i  \leftarrow  L_i + m_i^{(k)}  \;| \; i \in \; [k,n]$ then :

$\Omega^{(k)} = (G^{(k)})^-1 = \begin{pmatrix} 
1 & .. & 0 & 0 & 0 & 0 \\
0 & .. & .. & 0 & 0 & 0\\
0 & .. & 1 & 0 & 0 & 0\\
0 & .. &  m^{(k)}_{k+1} & 1 & 0 \\
0 & .. & .. & 0 & 1 & 0 \\
0 & .. &  m^{(k)}_{n} & 0 & 0 & 1
\end{pmatrix}$

to be sure you can compute $(G^{(k)}\Omega^{(k)})_{ij} = \sum_{p=1}^{p = n} g^{(k)}_{ip}  \omega^{(k)}_{pj} \;= \; \delta_{ij} = \; (I_n)_{ij} $ where 
$\begin{cases} g^{(k)}_{ij} = \delta_{ij} - m^{(k)}_{i} \delta_{ik}  \;\;i >= k \\ g^{(k)}_{ij} =0  \;\;i < k \end{cases}$ and $\begin{cases} \omega^{(k)}_{ij} = \delta_{ij} + m^{(k)}_{i} \delta_{ik}  \;\;i >= k \\ \omega^{(k)}_{ij} =0  \;\;i < k \end{cases}$

more than that we can show it in the notebook



#### 1.1 - Compute $G^{(k)}$ and $\Omega^{(k)}$

In [1]:
import numpy as np

In [2]:
def get_Gk(k,A, sign = -1):
    G = np.identity(A.shape[0]).astype(np.float64)
    j = np.argmax(abs(G[:,k]))
    G = np.array(permutation_matrix(G.shape[0],k,j) @ G)
    for i in range(k + 1, A.shape[0]):
        G[i,k] = + sign *  (A[i,k])/(A[k,k])
    return G
def get_Wk(k, A):
    return get_Gk(k,A, 1)

# We use permutation matrix to avoid akk = 0 
# This a permutation Matrix P that when multiplied by M, we permut for M the ith and jth line
def permutation_matrix(rang, i, j):
    P = np.identity(rang).astype(np.float64)
    tmp = P[j].copy()
    P[j] = P[i].copy()
    P[i] = tmp
    return P

In [3]:
# We take the same example as preivously in gauss method
import numpy as np
A = np.array([[-2,1,-1],[-3,-1,2],[-2,1,2]])
b = np.array([8, -11, -3]).reshape(3,1)
print("Matrix A\n",A)
print("\nVector b\n",b)

Matrix A
 [[-2  1 -1]
 [-3 -1  2]
 [-2  1  2]]

Vector b
 [[  8]
 [-11]
 [ -3]]


#### 1.2 - Confirming that  $G^{(k)-1} = \Omega^{(k)}$


In [4]:
print("G_k matrix :  \n\n",get_Gk(1,A), "\n")
print("W_k matrix :  \n\n",get_Wk(1,A), "\n")
print("W_k G_k product matrix wich is equal to identity : \n\n",get_Wk(1,A)@get_Gk(1,A), "\n")


G_k matrix :  

 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 1.]] 

W_k matrix :  

 [[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0. -1.  1.]] 

W_k G_k product matrix wich is equal to identity : 

 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]] 



#### 1.3 -U is equal to the triangular matrix obtained by gauss elimination method

We indeed had the identity by the product of $(G^{(k)}\Omega^{(k)})$,  now we are going to see that $U = G^{(n - 1)}..G^{(2)}G^{(1)}A$ where $U$ is the result of gauss elimination method

In [5]:
# we stack the matrix as we can do the lines operations in one time
stackA = np.hstack((A,b))
stackA

array([[ -2,   1,  -1,   8],
       [ -3,  -1,   2, -11],
       [ -2,   1,   2,  -3]])

In [6]:
# preivous method
def gauss(stackA):
    #we convert to np.float 64 because we do divisions that convert thing in float, so to avoid int cast we make it float
    T = np.array(stackA).astype(np.float64)
    for k in range(0, T.shape[0] - 1):
        p = 0
        for i in range(k + 1,T.shape[0]):
            m = T[i,k]/T[k,k]
            el = T[i] - (T[k] * m)
            T[i] +=  - (T[k] * m)
    return T

In [7]:
gauss(stackA)

array([[ -2. ,   1. ,  -1. ,   8. ],
       [  0. ,  -2.5,   3.5, -23. ],
       [  0. ,   0. ,   3. , -11. ]])

In [8]:
#new method
G1 = get_Gk(0,stackA)
G2 = get_Gk(1,G1)
G2 @ G1 @ stackA

array([[ -2. ,   1. ,  -1. ,   8. ],
       [  0. ,  -2.5,   3.5, -23. ],
       [  0. ,   0. ,   3. , -11. ]])

In [9]:
np.allclose(G2 @ G1 @ stackA ,gauss(stackA))

True

#### 1.4 - Comput L and U

In [10]:
# so to get U we define 
from functools import reduce
def get_U(A):
    G = np.array(A)
    list_G = []
    for i in range(A.shape[0] - 1):
        G = get_Gk(i, G)
        list_G.append(G)
    return reduce(lambda a,b:a@b, list_G[::-1]) @  A  
#and to get L we define 
def get_L(A):
    G = np.array(A)
    list_G = []
    for i in range(A.shape[0] - 1):
        G = get_Wk(i, G)
        list_G.append(G)
    return reduce(lambda a,b:a@b, list_G)



In [11]:
U = get_U(A)
U

array([[-2. ,  1. , -1. ],
       [ 0. , -2.5,  3.5],
       [ 0. ,  0. ,  3. ]])

In [12]:
L = get_L(A) 
L

array([[1. , 0. , 0. ],
       [1.5, 1. , 0. ],
       [1. , 0. , 1. ]])

In [13]:
# YES LU = A is RIGHT
L @ U  == A

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

#### 1.5 - solving an up and low triangular matrix functions

In [14]:
def solve_up(U, b0):
    n = len(b0)
    #we solve the last equation (line) in the matrix
    X = [b0[-1]/U[n - 1, n - 1]]
    #we go upper to solve the equations above using the last variable solved
    for k in range(n - 2, -1,-1):
        a = np.sum([X[(n - 1) - i] * U[k , i] for i in range(n - 1, k, -1)])
        term = (1/(U[k,k])) * (b0[k] - a)
        X.append(term)
    # we reverse W because, we solve it from down to up
    return np.array(X[::-1]).reshape(n,1)
def solve_down(L, b0):
    n = len(b0)
    #we solve the last equation (line) in the matrix
    X = [b0[0]/L[0, 0]]
    #we go upper to solve the equations above using the last variable solved
    for k in range(1, n):
        a = np.sum([[L[k,i]] * X[i] for i in range(k)])
        term = (1/(L[k,k])) * (b0[k] - a)
        X.append(term)
    # we reverse W because, we solve it from down to up
    return np.array(X).reshape(n,1)

#### 1.6 - Confirming the theory
 $Ax = b$ it's equivalent to solve $\begin{cases} Ly = b \\ Ux = y  \end{cases}$ 

In [15]:
Y = solve_down(L, b.astype(float))

In [16]:
X = solve_up(U,Y)

In [17]:
# we obtain as expexted  A X = b which mean that X is our solution
np.allclose(A @ X, b)

True

### 2 - Computing determinent  det(A)

#### 2.1 Leibinz Formula

In this part instead of using numpy for getting the determinant we are going to use [Leibniz Formula](https://en.wikipedia.org/wiki/Leibniz_formula_for_determinants) described by :

$det(A) = \sum_{\sigma \in Sn} sign(\sigma) \, \prod_{i = 1}^{i = n} a_{\sigma(i),i}$

where $\sigma$ is a permutation for $\{1,2,...,n\}$ in the permutation groupe $S_n$  and $sign$ return 1 -1 for even and odd permutations respectively



In [18]:
from itertools import permutations
import numpy as np

In [19]:
def signature(sigma):
    count = 0
    for i in range(len(sigma)):
        for j in range(i, len(sigma)):
            if sigma[i]> sigma[j]:
                count += 1
    if (count % 2 == 0):
        return 1
    else:
        return -1
                

In [20]:
def leibinz_formula(A):
    detA = 0
    for sigma in permutations(range(A.shape[0])):
        detA += signature(sigma) * np.product([A[sigma[i], i] for i in range(A.shape[0])])
    return detA

In [21]:
leibinz_formula(A)

15

#### 2.2 - determinent of a triangular 

When you have a triangular matrix $T_n$ so the formula for the det is : 

$det(T_n) = \prod_{i = 1}^{i = n}t_{i,i}$

you can check the [proof](https://proofwiki.org/wiki/Determinant_of_Triangular_Matrix)

there for $det(A) = det(L) * det(U) =  \prod_{i = 1}^{i = n}l_{i,i} *  \prod_{i = 1}^{i = n}u_{i,i} $

In [22]:
# diagonal matrix product 
def product_diag(T):
    return np.product([T[i,i] for i in range(T.shape[0])])

In [23]:
detA_li = leibinz_formula(A)
detA_np = np.linalg.det(A)
detA_lu = product_diag(L) * product_diag(U)

In [24]:
print("determinent with liebniz formula : ", detA_li)
print("determinent with numpy : ", detA_np)
print("determinent with factorization LU : ", detA_lu)

determinent with liebniz formula :  15
determinent with numpy :  15.000000000000007
determinent with factorization LU :  15.0


### 3 -Computing inverse matrix $A^{-1}$

#### - 3.1 Gauss Jordan elimination

The method that we have seen until now is Gauss method, where we operate transformations on $A$ until obtaining
the echlon form (the triangular matrix) but when we go further and the triangular matrix is identity, it usually called Gauss Jordan elimination.

The idea is by transorming $[A|I_n]$ to $[I_n|B]$ is to record the elementary operations performed on A in B such as $B = A^{-1}$, you can see it as if you had a system $AX = X^{'}$ and you find $X= BX^{'} $

Check more [details](https://en.wikipedia.org/wiki/Gaussian_elimination)

For our implementation we obtain the upper matrix $U$ then we operate transformations to make diagonal

#### - 3.2 Make diagonal out of Upper matrix

In [25]:
def diag(U):
    U = np.array(U)
    for i in range(1,U.shape[0]):
        for p in range(0, i):
            if (U[p, i] != 0):
                U[p] = (U[p]/U[p,i]) * U[i,i] - U[i]
    return U
        
    

#### - 3.2 Make identity out of diagonal

In [26]:
def make_id_fdiag(D):
    for i in range(D.shape[0]):
        D[i] = D[i] / (D[i,i])
    return D

#### - 3.3 getting the inverse $A^{-1}$

In [27]:
def get_inv(A):
    In = np.identity(A.shape[0])
    stackA = np.hstack((A,In))
    IB = make_id_fdiag(diag(get_U(stackA)))
    return IB[:,3:]

In [28]:
# testing 
get_inv(A) @ A

array([[ 1.00000000e+00,  1.38777878e-17, -2.77555756e-17],
       [ 0.00000000e+00,  1.00000000e+00,  1.11022302e-16],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [29]:
np.allclose(get_inv(A) @ A, np.identity(A.shape[0]))

True

Thank you ! hope that i have brought some value in LU Factorization